In [150]:
from PIL import Image
import numpy as np
import os
import glob
import rawpy
import imageio.v3 as iio
import random
from MTB import MTB_color
import cv2


In [189]:
def align_images_rgb(images, offsets):
    """
    Apply affine shift to a list of full-color (H, W, 3) images using given offsets.
    Assumes images[0] is reference and not shifted.
    """
    aligned = [images[0]]
    for i, (dx, dy) in enumerate(offsets):
        M = np.float32([[1, 0, dx], [0, 1, dy]])
        shifted = cv2.warpAffine(images[i + 1], M, (images[i + 1].shape[1], images[i + 1].shape[0]))
        aligned.append(shifted)
    return np.stack(aligned)

def load_images_and_exposures(image_paths):
    image_paths = [os.path.join(image_paths, f) for f in os.listdir(image_paths) if f.endswith('.tiff')]
    images = []
    exposures = []
    for p in image_paths:
        img = Image.open(p).convert('L')  # grayscale
        images.append(np.array(img).astype(np.float32))
        
        exposure = float(p.split('/')[-1].split('.')[0])
        exposures.append(1/exposure)
    return np.stack(images), np.array(exposures)

def load_rgb_images(path):
    image_paths = [
        os.path.join(path, f)
        for f in os.listdir(path) 
        if f.endswith('.tiff')
    ]
    exposures = []
    images = []
    for p in sorted(image_paths):
        img = Image.open(p).convert('RGB')
        img_np = np.array(img).astype(np.float32)
        images.append(img_np)
        # If the filename is "0.1.tiff", then exposure_time = 0.1
        filename = os.path.splitext(os.path.basename(p))[0]
        exposure_time = 1/ float(filename)
        exposures.append(exposure_time)
    images = np.stack(images, axis=0)  # shape (P, H, W, 3)
    exposures = np.array(exposures)    # shape (P,)
    return images, exposures



In [181]:
def mitsunaga_nayar_calibration(
        images, 
        exposure_times, 
        degree=5, 
        num_samples=1000, 
        clamp_min=0.01, 
        clamp_max=0.99, 
        random_seed=42
    ):
    """
    Solve for polynomial camera response function coefficients using Mitsunaga–Nayar method.
    
    images:  np.ndarray, shape (N, H, W) - single channel
             (If you have color images of shape (N,H,W,3), call this function once per channel.)
    exposure_times: np.ndarray of length N
    degree:         polynomial degree (often 3-5)
    num_samples:    how many random pixels to sample
    clamp_min:      ignore intensities below this threshold
    clamp_max:      ignore intensities above this threshold
    random_seed:    for reproducible pixel sampling

    Returns:
        a:             np.ndarray of shape (degree+1,), polynomial coefficients for f(I).
        logE_samples:  np.ndarray of shape (M,), the recovered log-radiance for each sampled pixel
        sample_coords: list of (row, col) for the chosen sample positions
    """
    # Ensure floating [0..1]
    images = images.astype(np.float64)
    if images.max() > 1.0:
        images /= 255.0

    N, H, W = images.shape
    
    np.random.seed(random_seed)

    # We will collect valid samples as we pick random pixel coords
    sample_coords = []
    # We'll pick up to num_samples pixels that meet clamp_min < I < clamp_max
    max_tries = num_samples * 10  # to avoid infinite loops if not enough valid
    tries = 0

    while len(sample_coords) < num_samples and tries < max_tries:
        r = np.random.randint(0, H)
        c = np.random.randint(0, W)
        # check if all N frames have intensities for this (r,c) in the valid range
        pixel_values = images[:, r, c]
        # we skip if any exposure is out of range or near 0/1
        if (pixel_values >= clamp_min).all() and (pixel_values <= clamp_max).all():
            sample_coords.append((r, c))
        tries += 1

    # If we didn't manage to find enough samples, we'll just use however many we got
    M = len(sample_coords)
    if M == 0:
        raise ValueError("No valid samples found. Consider lowering clamp_min or raising clamp_max.")

    K = degree
    # We'll build the system A x = b, size: (N*M) x (K+1 + M)
    A = np.zeros((N * M, (K + 1) + M), dtype=np.float64)
    b = np.zeros(N * M, dtype=np.float64)

    # Fill the system
    # Equation: sum_{k=0..K} a_k * I^k - L_j = log(t_i)
    for j, (r, c) in enumerate(sample_coords):
        for i in range(N):
            I_ij = images[i, r, c]  # single scalar
            row_idx = i + j * N

            # Fill polynomial part for a0..aK
            for k in range(K + 1):
                A[row_idx, k] = I_ij ** k

            # Fill -L_j (the logE for sample j)
            # logE_j index is (K+1)+j
            A[row_idx, (K+1) + j] = -1.0

            b[row_idx] = np.log(exposure_times[i])

    # Solve in least squares sense
    x, residuals, rank, svals = np.linalg.lstsq(A, b, rcond=None)

    # Extract polynomial coeffs: a0..aK
    a = x[:(K+1)]
    # Extract logE for each sample
    logE_samples = x[(K+1):]

    return a, logE_samples, sample_coords

def find_best_polynomial_degree(images, exposure_times, max_degree=10, num_samples=1000):
    """
    Finds the best polynomial degree (M) for Mitsunaga–Nayar calibration by minimizing reconstruction error.
    
    images: np.ndarray of shape (N, H, W, 3)
    exposure_times: np.ndarray of length N

    Returns:
        best_degree: int
        best_coeffs_rgb: list of np.ndarray, best polynomial coeffs for R, G, B
        errors: list of float, total combined error per degree
    """
    errors = []
    coeffs_all = []

    # Split channels
    channels = [images[..., c] for c in range(3)]

    for degree in range(1, max_degree + 1):
        coeffs_rgb = []
        error_rgb = []

        for ch in channels:
            a, logE, coords = mitsunaga_nayar_calibration(
                ch, exposure_times, degree=degree, num_samples=num_samples
            )
            coeffs_rgb.append(a)

            # Reproject using fitted polynomial
            reconstructed_logE = []
            for i in range(len(exposure_times)):
                I_vals = np.array([ch[i, r, c] for (r, c) in coords])
                f_I = apply_response_polynomial(I_vals, a)
                logE_est = f_I - np.log(exposure_times[i])
                reconstructed_logE.append(logE_est)

            # Stack shape: (N, M)
            reconstructed_logE = np.stack(reconstructed_logE, axis=0)
            # Compute mean squared error per pixel
            logE_mean = reconstructed_logE.mean(axis=0)
            mse = ((reconstructed_logE - logE_mean) ** 2).mean()
            error_rgb.append(mse)

        combined_error = sum(error_rgb)
        errors.append(combined_error)
        coeffs_all.append(coeffs_rgb)

        print(f"Degree {degree}: Error = {combined_error:.6f}")

    best_idx = np.argmin(errors)
    best_degree = best_idx + 1  # degrees start from 1
    best_coeffs_rgb = coeffs_all[best_idx]

    print(f"Best polynomial degree: {best_degree} with error {errors[best_idx]:.6f}")
    return best_degree, best_coeffs_rgb, errors



def apply_response_polynomial(I, a):
    """
    Evaluate the polynomial f(I) = a0 + a1*I + a2*I^2 + ...
    on intensity I (can be float or array).
    """
    # a is array of shape (K+1,)
    # If I is an array, we'll broadcast operations
    result = np.zeros_like(I, dtype=np.float64)
    for k, coeff in enumerate(a):
        result += coeff * (I ** k)
    return result

def reconstruct_hdr_color(
        images, 
        exposure_times, 
        coeffs_rgb, 
        weight_fn='linear', 
        logE_clip=(-20, 20)
    ):
    """
    Reconstruct a color HDR image from multi-exposure images using the Mitsunaga–Nayar
    polynomial coefficients for each channel.

    images: (N, H, W, 3) in [0..1] or [0..255]
    exposure_times: array of length N
    coeffs_rgb: list or tuple of length 3, each an array of shape (degree+1,) => [a_R, a_G, a_B]
    weight_fn: string indicating weighting approach: 'linear' or 'debevec_malik'
    logE_clip: clamp log-radiances to avoid overflow in exp()

    returns HDR image of shape (H, W, 3) in float64
    """
    images = images.astype(np.float64)
    if images.max() > 1.0:
        images /= 255.0

    N, H, W, C = images.shape
    hdr = np.zeros((H, W, C), dtype=np.float64)
    weight_sum = np.zeros((H, W, C), dtype=np.float64)

    for i in range(N):
        # loop over each channel separately
        for ch in range(C):
            I_ch = images[i, :, :, ch]
            # polynomial => log(E) + log(t_i)
            f_I = apply_response_polynomial(I_ch, coeffs_rgb[ch])
            logE_ch = f_I - np.log(exposure_times[i])

            # clamp logE to avoid np.exp overflow or underflow
            logE_ch_clamped = np.clip(logE_ch, logE_clip[0], logE_clip[1])
            E_ch = np.exp(logE_ch_clamped)

            # weighting
            if weight_fn == 'linear':
                # clamp intensities so we don't rely on pure blacks/whites
                w_ch = np.clip(I_ch, 0.01, 0.99)
            elif weight_fn == 'debevec_malik':
                # w(I) = I*(1 - I) in [0..1], ignoring extremes
                w_ch = I_ch * (1.0 - I_ch)
            else:
                # fallback to a simple clamp
                w_ch = np.clip(I_ch, 0.01, 0.99)

            hdr[..., ch] += w_ch * E_ch
            weight_sum[..., ch] += w_ch

    valid = weight_sum > 0
    hdr_out = np.zeros_like(hdr, dtype=np.float64)
    hdr_out[valid] = hdr[valid] / weight_sum[valid]

    return hdr_out



def tone_map(hdr_image, gamma=2.2, exposure=1e-7):
    img_tm = 1.0 - np.exp(-hdr_image * exposure)
    img_tm = np.clip(img_tm, 0, 1)
    img_tm = img_tm ** (1 / gamma)
    return (img_tm * 255).astype(np.uint8)

In [164]:
def compute_mitsunaga_nayar_error(images, exposure_times, coeffs, sample_coords):
    """
    Computes Mitsunaga-Nayar error ε based on the provided camera response coefficients.
    
    images: (N, H, W) single-channel image stack
    exposure_times: array of length N
    coeffs: array of length (M+1,), the c_m coefficients
    sample_coords: list of (row, col)
    
    Returns:
        scalar error ε
    """
    images = images.astype(np.float64)
    if images.max() > 1.0:
        images /= 255.0

    N, H, W = images.shape
    M = len(coeffs) - 1
    P = len(sample_coords)

    error = 0.0

    for j in range(P):
        r, c = sample_coords[j]
        for i in range(N):
            Z_ij = images[i, r, c]
            Z_ij1 = images[i, r, c]  # same pixel across images

            # Compute polynomial values
            poly_ij = sum(coeffs[m] * (Z_ij ** m) for m in range(M + 1))
            poly_ij1 = sum(coeffs[m] * (Z_ij1 ** m) for m in range(M + 1))

            # Exposure ratio R = t_j / t_{j+1} → in practice: R_{j, j+1}
            if i < N - 1:
                R = exposure_times[i] / exposure_times[i + 1]
            else:
                R = exposure_times[i - 1] / exposure_times[i]  # fallback for last frame

            term = poly_ij - R * poly_ij1
            error += term ** 2

    return error
def find_best_degree_using_mitsunaga_error(images, exposure_times, max_degree=10, num_samples=1000):
    """
    Tries multiple degrees (1..max_degree) and computes the Mitsunaga-Nayar error.
    Returns the best degree with minimal combined RGB error.
    """
    errors = []
    coeffs_all = []

    # Extract RGB channels
    channels = [images[..., c] for c in range(3)]

    for degree in range(1, max_degree + 1):
        coeffs_rgb = []
        error_rgb = []
        sample_coords = None

        for ch in channels:
            a, _, coords = mitsunaga_nayar_calibration(
                ch, exposure_times, degree=degree, num_samples=num_samples
            )
            coeffs_rgb.append(a)
            if sample_coords is None:
                sample_coords = coords  # use same coords across channels
            error = compute_mitsunaga_nayar_error(ch, exposure_times, a, sample_coords)
            error_rgb.append(error)

        combined_error = sum(error_rgb)
        errors.append(combined_error)
        coeffs_all.append(coeffs_rgb)
        print(f"Degree {degree}: Error = {combined_error:.4f}")

    best_idx = np.argmin(errors)
    best_degree = best_idx + 1
    print(f"✅ Best polynomial degree: {best_degree} with error {errors[best_idx]:.4f}")
    return best_degree, coeffs_all[best_idx], errors


In [188]:
data_path = "../data/test/"
output_path = "../data/output/mitsunaga/"
os.makedirs(output_path, exist_ok=True)

# 1) Load images, exposures
images, exposure_times = load_rgb_images(data_path)  

ValueError: need at least one array to stack

In [ ]:
# MTB
offsets = MTB_color(images)
images_aligned = align_images_rgb(images, offsets)

In [165]:
best_degree, best_coeffs_rgb, errors = find_best_degree_using_mitsunaga_error(images_aligned, exposure_times, max_degree=10, num_samples=1000)

Degree 1: Error = 11326661.8582
Degree 2: Error = 9890977.8825
Degree 3: Error = 9681801.7641
Degree 4: Error = 9670589.3337
Degree 5: Error = 9701016.4438
Degree 6: Error = 9753503.0029
Degree 7: Error = 9771752.0391
Degree 8: Error = 9739253.7955
Degree 9: Error = 9729465.4019
Degree 10: Error = 8690246.7014
✅ Best polynomial degree: 10 with error 8690246.7014


In [170]:
degree = 4
coeffs_rgb = []

# We'll handle R, G, B channels separately
for ch in range(3):
    # single_channel shape => (N, H, W)
    single_channel = images_aligned[:, :, :, ch]

    a_ch, logE_samps, sample_coords = mitsunaga_nayar_calibration(
        single_channel,
        exposure_times,
        degree=degree,
        num_samples=2000,
        clamp_min=0.01,
        clamp_max=0.99
    )
    coeffs_rgb.append(a_ch)

In [171]:
hdr_result = reconstruct_hdr_color(
    images_aligned,
    exposure_times,
    coeffs_rgb,
    weight_fn='debevec_malik',   # or 'linear'
    logE_clip=(-20, 20)          # safe exponent clamp
)

In [172]:
tone_mapped = tone_map(hdr_result, gamma=1.7, exposure=0.2)
iio.imwrite(os.path.join(output_path, "tone_mapped.jpg"), tone_mapped)

In [173]:
hdr_tonemapped = (hdr_result / (hdr_result + 1.0))  # naive
hdr_tonemapped = np.clip(hdr_tonemapped, 0, 1)
iio.imwrite(os.path.join(output_path,"output_ldr.png"), (hdr_tonemapped*255).astype(np.uint8))

In [148]:
iio.imwrite(os.path.join(output_path, "output.hdr"),
            hdr_result.astype(np.float32))